In [106]:
from decimal import Decimal
from math import sqrt
from typing import Any


def calcAvg(x):
    sum = 0
    for i in x:
        sum = sum+i
    return sum/len(x)

def calcSigma(x):
    avg = calcAvg(x)
    sum  = 0
    for i in x:
        sum = (sum+(avg-i)**2)
    return sqrt(sum/(len(x)-1))

def calcUncertainty(x, d, c):
    sigma = calcSigma(x)
    delta_a = sigma*c
    delta_b = d/c
    return sqrt(delta_a**2 + delta_b**2)

class ExpData:
    def __init__(self, avg, uncertainty):
        self.avg = avg
        self.u = uncertainty
    
    def __str__(self):
        return f"avg: {self.avg}, u: {self.u}, ur: {self.u/self.avg}"

    def __repr__(self):
        return f"avg: {self.avg}, u: {self.u}, ur: {self.u/self.avg}"
    
    def __add__(self, other):
        avg = self.avg + other.avg
        u = sqrt(self.u**2 + other.u**2)
        return ExpData(avg, u)

    def __sub__(self, other):
        avg = self.avg - other.avg
        u = sqrt(self.u**2 + other.u**2)
        return ExpData(avg, u)

    def __mul__(self, other):
        avg = self.avg*other.avg
        ur1 = self.u/self.avg
        ur2 = other.u/other.avg
        ur = sqrt(ur1**2 + ur2**2)
        u = ur*avg
        return ExpData(avg, u)
    
    def __truediv__(self, other):
        avg = self.avg/other.avg
        ur1 = self.u/self.avg
        ur2 = other.u/other.avg
        ur = sqrt(ur1**2 + ur2**2)
        u = ur*avg
        return ExpData(avg, u)
    
    def __pow__(self, other):
        avg = self.avg**other
        ur1 = self.u/self.avg
        ur = abs(ur1*other)
        u = ur*avg
        return ExpData(avg, u)


def genData(x:list, d, c=1.05):
    avg = calcAvg(x)
    u = calcUncertainty(x, d, c)
    return ExpData(avg, u)

def cons(x) :
    return ExpData(x, 0)


## 凸透镜
### 自准法

In [107]:
delta = float('0.05')
c = float('1.05')
n = 6

In [108]:
from decimal import Decimal

x_1ll = ['20.95', '20.45', '20.45', '20.50', '20.80', '20.45']
x_1rr = ['25.80', '25.25', '25.25', '25.40', '25.65', '25.35']
x_2ll = ['20.05', '20.15', '20.05', '19.90', '20.20', '19.85']
x_2rr = ['24.90', '25.00', '24.90', '24.80', '25.10', '24.80']

x_1ll = [float(x) for x in x_1ll]
x_1rr = [float(x) for x in x_1rr]
x_2ll = [float(x) for x in x_2ll]
x_2rr = [float(x) for x in x_2rr]

x_1l = genData(x_1ll, delta, c)
x_1r = genData(x_1rr, delta, c)
x_2l = genData(x_2ll, delta, c)
x_2r = genData(x_2rr, delta, c)

pos = ExpData((float('1.20')+float('6.60'))/2+float('22.96')/10, 0)

x = ((x_1l+x_1r)/cons(2)+(x_2l+x_2r)/cons(2))/cons(2)

f = x-pos
f

avg: 16.554000000000002, u: 0.09806416738052835, ur: 0.0059238955769317595

In [109]:
print(0.05/1.05)
print(pos)
print('x_1l', x_1l, 'Sigma:', calcSigma(x_1ll))
print('x_1r', x_1r, 'Sigma:', calcSigma(x_1rr))
print('x_2l', x_2l, 'Sigma:', calcSigma(x_2ll))
print('x_2r', x_2r, 'Sigma:', calcSigma(x_2rr))
print(x)

0.047619047619047616
avg: 6.196, u: 0, ur: 0.0
x_1l avg: 20.599999999999998, u: 0.2349203560701908, ur: 0.01140390078010635 Sigma: 0.21908902300206665
x_1r avg: 25.45, u: 0.24185754835469797, ur: 0.009503243550282828 Sigma: 0.2258317958127242
x_2l avg: 20.033333333333335, u: 0.1511541388654144, ur: 0.007545131723731167 Sigma: 0.13662601021279416
x_2r avg: 24.916666666666668, u: 0.13166272705722448, ur: 0.0052841228250391096 Sigma: 0.11690451944500146
avg: 22.75, u: 0.09806416738052835, ur: 0.004310512851891356


### 共轭法

In [110]:
x_1ll = ['27.00', '27.00', '27.50', '27.80', '28.10', '28.50']
x_1rr = ['31.80', '31.80', '32.30', '32.60', '33.00', '33.30']
x_2ll = ['54.05', '54.40', '53.45', '52.70', '53.10', '53.10']
x_2rr = ['58.95', '59.20', '58.30', '57.50', '58.00', '58.00']

x_1ll = [float(x) for x in x_1ll]
x_1rr = [float(x) for x in x_1rr]
x_2ll = [float(x) for x in x_2ll]
x_2rr = [float(x) for x in x_2rr]

x_1l = genData(x_1ll, delta, c)
x_1r = genData(x_1rr, delta, c)
x_2l = genData(x_2ll, delta, c)
x_2r = genData(x_2rr, delta, c)

In [111]:
pos_t = (float('78.45')+float('82.80'))/2
pos_t = ExpData(pos_t, 0)
pos_t

avg: 80.625, u: 0, ur: 0.0

In [112]:
D = pos_t - pos
x_1 = (x_1l+x_1r)/ExpData(2, 0)
x_2 = (x_2l+x_2r)/ExpData(2, 0)
d = x_2-x_1
f = (D**2-d**2)/(ExpData(4, 0)*D)
f

avg: 16.364923735204023, u: 0.11444483333224093, ur: 0.006993300743959387

In [113]:
print(D)
print(d)
print('x_1l', x_1l, 'Sigma:', calcSigma(x_1ll))
print('x_1r', x_1r, 'Sigma:', calcSigma(x_1rr))
print('x_1', x_1)
print('x_2', x_2)
print('x_2l', x_2l, 'Sigma:', calcSigma(x_2ll))
print('x_2r', x_2r, 'Sigma:', calcSigma(x_2rr))
print(x)

avg: 74.429, u: 0.0, ur: 0.0
avg: 25.83749999999999, u: 0.6593528398711458, ur: 0.025519219733764722
x_1l avg: 27.650000000000002, u: 0.6344092320388673, ur: 0.022944276023105507 Sigma: 0.602494813255683
x_1r avg: 32.46666666666667, u: 0.6512738085445661, ur: 0.02005976823032544 Sigma: 0.618600571181975
x_1 avg: 30.058333333333337, u: 0.4545967024166282, ur: 0.015123815993899467
x_2 avg: 55.89583333333333, u: 0.4775856002834176, ur: 0.008544207533956037
x_2l avg: 53.46666666666667, u: 0.6770044857282279, ur: 0.012662178660752392 Sigma: 0.6431692364118982
x_2r avg: 58.324999999999996, u: 0.6738077980375019, ur: 0.011552641200814435 Sigma: 0.6401171767731293
avg: 22.75, u: 0.09806416738052835, ur: 0.004310512851891356


## 凹透镜
### 物距像距法

In [114]:
x_1ll = ['59.20', '58.95', '59.30', '58.75', '59.30', '59.10']
x_1rr = ['64.10', '63.85', '64.20', '63.65', '64.20', '64.00']
x_2ll = ['59.70', '59.10', '59.65', '58.90', '59.55', '59.15']
x_2rr = ['64.60', '64.00', '64.55', '63.80', '64.45', '64.15']

x_1ll = [float(x) for x in x_1ll]
x_1rr = [float(x) for x in x_1rr]
x_2ll = [float(x) for x in x_2ll]
x_2rr = [float(x) for x in x_2rr]

x_1l = genData(x_1ll, delta, c)
x_1r = genData(x_1rr, delta, c)
x_2l = genData(x_2ll, delta, c)
x_2r = genData(x_2rr, delta, c)

A_1 = (float('71.40')+float('76.30'))/2
A_2 = (float('91.75')+float('96.65'))/2
A_1 = ExpData(A_1, 0)
A_2 = ExpData(A_2, 0)

In [115]:
x_1 = (x_1l + x_1r) / ExpData(2, 0)
x_2 = (x_2l + x_2r) / ExpData(2, 0)
x = (x_1 + x_2) / ExpData(2, 0)
s = A_1 - x
s_ = A_2 - x
f = (s**(-1)+s_**(-1))**(-1)
f, s**(-1), s, x

(avg: 8.858878635346754, u: 0.07926973930884156, ur: 0.008948055681963723,
 avg: 0.08213552361396306, u: 0.0010002942954861717, ur: 0.012178583047544137,
 avg: 12.174999999999997, u: 0.1482742486038498, ur: 0.012178583047544135,
 avg: 61.675, u: 0.1482742486038498, ur: 0.0024041223932525302)

In [116]:
print('A_1', A_1)
print('A_2', A_2)
print('x_1l', x_1l)
print('x_1r', x_1r)
print('x_2l', x_2l)
print('x_2r', x_2r)
print('x_1', x_1)
print('x_2', x_2)
print('x', x)
print('s', s)
print('s_', s_)

A_1 avg: 73.85, u: 0, ur: 0.0
A_2 avg: 94.2, u: 0, ur: 0.0
x_1l avg: 59.1, u: 0.23256197818247196, ur: 0.003935058852495296
x_1r avg: 64.0, u: 0.23256197818247398, ur: 0.003633780909101156
x_2l avg: 59.34166666666667, u: 0.35370107251200894, ur: 0.005960416893897075
x_2r avg: 64.25833333333333, u: 0.3442229636386051, ur: 0.005356861060385503
x_1 avg: 61.55, u: 0.16444615181898456, ur: 0.002671749014118352
x_2 avg: 61.8, u: 0.2467761624389043, ur: 0.0039931417870372864
x avg: 61.675, u: 0.1482742486038498, ur: 0.0024041223932525302
s avg: 12.174999999999997, u: 0.1482742486038498, ur: 0.012178583047544135
s_ avg: 32.525000000000006, u: 0.1482742486038498, ur: 0.004558777820256718
